In [23]:
import pymysql
import MySQLdb
from sqlalchemy import create_engine

# import transcript notebook
%run transcript.ipynb

# DB configuration
DB_USERNAME = "tikunli_lab415"
DB_PASS = "4]1#lrQ(ucGR"
DB_HOST = "31.220.17.169"
DB_NAME = "tikunli_bisli"
DB_CHARSET = "utf8mb4"

# connect to DB and get cursor
conn = pymysql.connect(host = DB_HOST,
                             user = DB_USERNAME,
                             password = DB_PASS,
                             db = DB_NAME,
                             charset = DB_CHARSET)

In [24]:
'''
prepareDF: prepare the dataframe with the replacements
'''
def prepareDF():
    exclude_pairs = {'yegia': 'hegia', 'yishtatfu': 'hishtatfu'}
    main(threshold = 0.7, exclude_pairs = exclude_pairs, doPrint = False)
    print("success");

'''
commit_whole_db: UPLOAD data with replaced words to MySQL DB
'''
def commit_whole_db():
    engine = create_engine('mysql+mysqldb://{0}:{1}@{2}/{3}?charset={4}'.format(DB_USERNAME, DB_PASS, DB_HOST, DB_NAME, DB_CHARSET))
    df.to_sql(name = 'orig_after_repl_2', con = engine, if_exists='replace', index=True, index_label='cid')
    print("uploaded!")
    
    
'''
xlsExport: EXPORT all data with replaced words to Excel
'''
def xlsExport():
    writer = pd.ExcelWriter('PythonExport.xlsx')
    df.to_excel(writer,'Sheet5')
    writer.save()

    
'''
doSyntaxColumn
'''
def doSyntaxColumn():
    cursor = conn.cursor()
    edit_cursor = conn.cursor()
    query = ""

    cursor.execute("SELECT * FROM questions")
    for row in cursor:
        syntax = re.split("_", row[1], 2)[2]
        syntax = re.sub(r'\d', '', syntax)
        syntax = re.sub(r'_$', '', syntax)

        query = "UPDATE questions SET syntax=%s WHERE qid=%s;"
        edit_cursor.execute(query, (norm, row[0]))
        print (row[0], "-->", syntax)

        
'''
qtextColPurify
'''
def qtextColPurify():
    cursor = conn.cursor()
    edit_cursor = conn.cursor()

    query = ""

    cursor.execute("SELECT qid, qtext, lang FROM questions")
    for row in cursor:
        lang = row[2]
        norm = row[1].lower()

        # trim and remove multiple spaces
        norm = norm.strip()
        norm = re.sub("([\s]+)", ' ', norm)

        if lang == "HE": 
            norm = re.sub(r'[^\w\s]|_|', '', norm)
            norm = re.sub('-', ' ', norm)
        else:
            norm = re.sub(r'[^\w\'\s]|_|', '', norm)

        query = "UPDATE questions SET qtext=%s WHERE qid=%s;"
        edit_cursor.execute(query, (norm, row[0]))

        print (row[0], "-->", norm)

'''
question_words_num
'''    
def question_words_num():
    cursor = conn.cursor()
    edit_cursor = conn.cursor()
    query = ""

    cursor.execute("SELECT qid, qtext, words_num FROM questions")
    for row in cursor:
        # split text by space
        words = row[1].split()
        count = len(words)

        query += "UPDATE questions SET words_num="+ str(count) +" WHERE qid="+ str(row[0]) +";"
        # print (row[0], "-->", count)

    print(query)
   

'''
answer_words_num
'''
def answer_words_num():
    cursor = conn.cursor()
    edit_cursor = conn.cursor()

    query = ""

    cursor.execute("SELECT aid, atext, words_num FROM answers where atext != 'CORRECT' and words_num = 0")
    for row in cursor:
        # split answer by space
        words = row[1].split()
        count = len(words)

        query += "UPDATE answers SET words_num="+ str(count) +" WHERE aid="+ str(row[0]) +";"
        #print (row[0], "-->", count)

    print(query)
    
    # if the answer is correct then the words number should be equal to maximum (as in the num of words in question)
    query = 'UPDATE `answers` JOIN questions using(qid) SET answers.words_num = questions.words_num WHERE atext = "CORRECT"'
    edit_cursor.execute(query)
    
    
'''
correctWordsNum
'''
def correctWordsNum():
    cursor = conn.cursor()
    edit_cursor = conn.cursor()

    query = ""

    cursor.execute("SELECT aid, qid, atext, qtext FROM `answers` JOIN questions USING(qid) WHERE atext != 'CORRECT'")
    for row in cursor:
        correct = 0

        # split answer by space
        answer = row[2].split()
        question = row[3].split()
        for word in answer:
             if word in question:
                    correct += 1
                    question.remove(word)

        query += "UPDATE answers SET correct_words_num="+ str(correct) +" WHERE aid="+ str(row[0]) +";"
        #print (row[0], "-->", count)

    print(query)

    # if the answer is correct then the correct words number should be equal to maximum (as in the num of words in question)
    query = 'UPDATE `answers` SET correct_words_num = words_num WHERE atext = "CORRECT"'
    edit_cursor.execute(query)

    
'''
correctPairsNum
'''
def correctPairsNum():
    cursor = conn.cursor()
    edit_cursor = conn.cursor()

    query = ""

    cursor.execute("SELECT aid, qid, atext, qtext FROM `answers` JOIN questions USING(qid) WHERE atext != 'CORRECT'")
    for row in cursor:
        correct = 0

        # split answer by space
        answer = row[2].split()
        question = " " + row[3] + " "
    
        for i in range(len(answer) - 1):
            pair = " " + answer[i] + " " + answer[i+1] + " "
            if pair in question:
                correct += 1

        query += "UPDATE answers SET correct_pairs_num="+ str(correct) +" WHERE aid="+ str(row[0]) +";"
        #print (row[0], "-->", count)

    print(query)

    # if the answer is correct then the correct pairs number should be equal to maximum (as in the num of words in question)
    query = 'UPDATE `answers` SET correct_pairs_num = words_num - 1 WHERE atext = "CORRECT"'
    edit_cursor.execute(query)

1. add 30+ missing children from 2 excels
   Don't forget to update `excel` column
2. transcription russian -> english
3. function words: divide into groups add cols to qustions and answers
4. is the word after fnction word right? - another col in answers - for each pair - how many correct?
    be ... le ... 
    
    Q new cols (#groups): how many function words from each group
    A new cols (#groups * 3): how many appear, how many correct, how many correct pairs
                                                    Q: ba bayit yesh kelev
                                                    A: ba keleb ba bayit = 1 correct pair
                                                    
5. Bonus: capture roots / suffixes: ספירת עיצורים ותנועות במקום אותיות                                                  
            a. if root is correct -> count as correct
            b. if suffix is correct - Bonus!!